In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, UpSampling2D, BatchNormalization, Activation
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import random

# Define paths
data_dir = "..//cv_p3_images_split"
train_gray_dir = os.path.join(data_dir, "train/grayscale")
train_color_dir = os.path.join(data_dir, "train/colored")
val_gray_dir = os.path.join(data_dir, "validation/grayscale")
val_color_dir = os.path.join(data_dir, "validation/colored")

# Image dimensions
IMG_HEIGHT, IMG_WIDTH = 256, 256  # Resize all images to 256x256

# Utility to preprocess images
def preprocess_image(image_path, target_size):
    image = load_img(image_path, target_size=target_size, color_mode="rgb")
    image = img_to_array(image) / 255.0  # Normalize to [0, 1]
    return image

def load_dataset(gray_folder, color_folder, target_size, fraction=1.0):
    gray_images = []
    color_images = []
    
    filenames = os.listdir(gray_folder)
    if fraction < 1.0:
        filenames = random.sample(filenames, int(len(filenames) * fraction))  # Sample fraction of filenames
    
    for filename in filenames:
        gray_path = os.path.join(gray_folder, filename)
        color_path = os.path.join(color_folder, filename)
        gray_images.append(preprocess_image(gray_path, target_size)[..., 0:1])  # Extract grayscale channel
        color_images.append(preprocess_image(color_path, target_size))
    
    return np.array(gray_images), np.array(color_images)

train_gray, train_color = load_dataset(train_gray_dir, train_color_dir, (IMG_HEIGHT, IMG_WIDTH), 0.25)
val_gray, val_color = load_dataset(val_gray_dir, val_color_dir, (IMG_HEIGHT, IMG_WIDTH), 0.25)

# Define the colorization model
def build_model():
    inputs = Input(shape=(IMG_HEIGHT, IMG_WIDTH, 1))

    # Encoder
    x = Conv2D(32, (3, 3), padding="same", strides=2)(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(64, (3, 3), padding="same", strides=2)(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    # Decoder
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(32, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = UpSampling2D((2, 2))(x)
    x = Conv2D(3, (3, 3), padding="same")(x)
    outputs = Activation("sigmoid")(x)

    return Model(inputs, outputs)

# Training with RMSprop optimizer
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001)

model_mse = build_model()
model_mse.compile(optimizer=optimizer, loss="mse", metrics=["accuracy"])
model_mse.fit(train_gray, train_color, validation_data=(val_gray, val_color), epochs=20, batch_size=16)
model_mse.save("Final.h5")


Epoch 1/20
93/93 ━━━━━━━━━━━━━━━━━━━━ 31s 321ms/step - accuracy: 0.4739 - loss: 0.0198 - val_accuracy: 0.6205 - val_loss: 0.0627
Epoch 2/20
93/93 ━━━━━━━━━━━━━━━━━━━━ 30s 320ms/step - accuracy: 0.5750 - loss: 0.0109 - val_accuracy: 0.6247 - val_loss: 0.0498
Epoch 3/20
93/93 ━━━━━━━━━━━━━━━━━━━━ 29s 313ms/step - accuracy: 0.5702 - loss: 0.0107 - val_accuracy: 0.6264 - val_loss: 0.0403
Epoch 4/20
93/93 ━━━━━━━━━━━━━━━━━━━━ 30s 317ms/step - accuracy: 0.5700 - loss: 0.0104 - val_accuracy: 0.6172 - val_loss: 0.0277
Epoch 5/20
93/93 ━━━━━━━━━━━━━━━━━━━━ 30s 320ms/step - accuracy: 0.5805 - loss: 0.0105 - val_accuracy: 0.6238 - val_loss: 0.0188
Epoch 6/20
93/93 ━━━━━━━━━━━━━━━━━━━━ 29s 314ms/step - accuracy: 0.5742 - loss: 0.0098 - val_accuracy: 0.6177 - val_loss: 0.0095
Epoch 7/20
93/93 ━━━━━━━━━━━━━━━━━━━━ 29s 317ms/step - accuracy: 0.5637 - loss: 0.0102 - val_accuracy: 0.6053 - val_loss: 0.0149
Epoch 8/20
93/93 ━━━━━━━━━━━━━━━━━━━━ 30s 319ms/step - accuracy: 0.5987 - loss: 0.0098 - val_accu